# Playlist Generation

### Goal: 

Step 1: From a seed song find all songs from all related artists

### Import Libraries and create spotify credentials manager

In [1]:
#importing module spotipy to make api calls to spotify
import spotipy
# module to extract song from url
import urllib
# OAuth Credenials
from spotipy.oauth2 import SpotifyClientCredentials
from multiprocessing import Process, Queue
import pandas as pd 
import numpy as np
from sklearn import datasets, linear_model
#from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

client_credentials_manager = SpotifyClientCredentials(client_id='4a1a457d14c0445a96e7021a5523905b', client_secret='45b186d601544fdcbb4fe07c7b23cce5')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


### Read in csv that has the track ids from all Spotify playlists

In [2]:
track_info = pd.read_csv("kmeans_output.csv")

In [3]:
track_ids = list(track_info['id'])

In [4]:
len(track_ids)
print(track_ids[:15])

['1tPGSiwtvmMVAqmniJ7oHp', '1tpShM80HMT9vKjRiA3EzW', '1TPUbLdzpfgF7wrPEXIAAB', '1tQGRq2WOBXjL3JWdWMONg', '1TqmZVIyeD4aN4Wa0wbnfy', '1TquBahz02HCaxj8jy5tTo', '1Tr4K5MU5XYE44umXGDndd', '1tRlCCMVLSVQrrN9PPgtf0', '1TScJLsDWzDVBFpCU5vJDo', '1TSjS0q24cPW2S4pYj2wEK', '1Tt4sE4pXi57mTD1GCzsqm', '1TtdHZ4KvN56LCt9evBUaL', '1TTiZwoorWMR4KmaI0irjU', '1TtJJ7YP6JCQ3aH8PQpreD', '1TTOAw90zzNWKnUgzzcvZW']


In [5]:
track_result = sp.tracks(["1tpShM80HMT9vKjRiA3EzW"])

### Input a seed track

In [59]:
seed_id = track_ids[1]
track_info = sp.tracks([seed_id])
name = track_info['tracks'][0]['name']
seed_preview_url = track_info['tracks'][0]['preview_url']
print(name)

Merry Christmas Baby - Live


In [30]:
track_info['tracks'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4TKTii6gnOnUXQHyuo9JaD'},
  'href': 'https://api.spotify.com/v1/artists/4TKTii6gnOnUXQHyuo9JaD',
  'id': '4TKTii6gnOnUXQHyuo9JaD',
  'name': 'Sheryl Crow',
  'type': 'artist',
  'uri': 'spotify:artist:4TKTii6gnOnUXQHyuo9JaD'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/6PAt558ZEZl0DmdXlnjMgD'},
  'href': 'https://api.spotify.com/v1/artists/6PAt558ZEZl0DmdXlnjMgD',
  'id': '6PAt558ZEZl0DmdXlnjMgD',
  'name': 'Eric Clapton',
  'type': 'artist',
  'uri': 'spotify:artist:6PAt558ZEZl0DmdXlnjMgD'}]

### Get the track's artists

In [31]:
# Get artist
artist_names = []
artist_ids = []
for each in track_info['tracks'][0]['artists']:
    artist_names.append(each['name'])
    artist_ids.append(each['id'])
print(artist_names)
print(artist_ids)

['Sheryl Crow', 'Eric Clapton']
['4TKTii6gnOnUXQHyuo9JaD', '6PAt558ZEZl0DmdXlnjMgD']


### Get related artists

In [35]:
all_related_artists_names = []
all_related_artists_ids = []

for each in artist_ids:
    related = sp.artist_related_artists(each)
    for related_artist in related['artists']:
        all_related_artists_names.append(related_artist['name'])
        all_related_artists_ids.append(related_artist['id'])

In [40]:
all_related_artists_ids = np.array(all_related_artists_ids)

In [41]:
len(all_related_artists_ids)

40

In [43]:
all_related_artists_ids_unique = np.unique(all_related_artists_ids)
len(all_related_artists_ids_unique)

40

In [48]:
all_related_albums = []
for each in all_related_artists_ids_unique:    
    albums = sp.artist_albums(each, album_type=None, country=None, limit=50, offset=0)
    for each in albums['items']:
        album_id = each['id']
        all_related_albums.append(album_id)

In [49]:
len(all_related_albums)

1906

In [52]:
all_related_albums[0:10]

['5JMqUlAUeSP9RFinIKjFiq',
 '1SpttuqXRFeIyiEGHER2l7',
 '4GhCTHZu2jIKfrwt5aA5PP',
 '5QLk6oXIfWF1ReUWgwALO7',
 '5t8dPRekIWIIqEVzTyeDB5',
 '7iGU0V2aK7WLVzlQh3nnLK',
 '3T8j78AeHjwntZBYVCVsIW',
 '3XG0Fpp3cYCB6jFEkmeHG1',
 '6Rk3I1FhR2huhQqizJDTlT',
 '2S1E8X1iPWc8NZxynrYT4K']

In [57]:
sp.album_tracks(all_related_albums[1], limit=50, offset=0)

all_related_songs = []
all_related_preview_urls = []
for index, each_album in enumerate(all_related_albums):
    album_tracks = sp.album_tracks(each_album, limit=50, offset=0)
    print(index)
    for each in album_tracks['items']:
        all_related_songs.append(each['id'])
        all_related_preview_urls.append(each['preview_url'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Total number of related songs for a seed track

In [58]:
len(all_related_songs)

31967

### Test to make sure the songs sound related

In [66]:
from IPython.core.display import HTML
link = '<a href=' + str(seed_preview_url) + '>preview input song</a>'
HTML(link)


In [67]:
link = '<a href=' + str(all_related_preview_urls[0]) + '>preview related song</a>'
HTML(link)

### Result: They sound related but a playlist from an input Christmas song should probably have Christmas songs!

# Appendix: Code from dataset generation for reference

In [76]:
def get_popularity(numLoops, track_ids):
    # store all tracks of all playlists 
    print('Starting')
    track_ids_all= []
    popularity = []
    preview_urls_all =[]
    artists_all = []
    names_all = []
    track_numbers_all = []
    # set parameters
    offset=0
    limit=50
    aud_features = np.empty((limit*numLoops, 13))
    for loopIndex in range(numLoops):
        start = offset + limit*loopIndex # start 
        tracks = track_ids[start:start+limit]
        tracks_list = sp.tracks(tracks)
        features = sp.audio_features(tracks=tracks)
        for index, track in enumerate(tracks_list['tracks']):

            try:
                if (start+index)%100==0: 
                    print(index+1+start)
                preview_urls_all.append(track['preview_url'])
                track_ids_all.append(track['id'])
                popularity.append(track['popularity'])
                names_all.append(track['name'])
                artists_all.append(track['artists'][0]['name'])
                track_numbers_all.append(track['track_number'])
                songData = features[index]
                aud_features[index+start][0] = songData['acousticness']
                aud_features[index+start][1] = songData['danceability']
                aud_features[index+start][2] = songData['energy']
                aud_features[index+start][3] = songData['instrumentalness']
                aud_features[index+start][4] = songData['key']
                aud_features[index+start][5] = songData['liveness']
                aud_features[index+start][6] = songData['loudness']
                aud_features[index+start][7] = songData['mode']
                aud_features[index+start][8] = songData['tempo']
                aud_features[index+start][9] = songData['time_signature']
                aud_features[index+start][10] = songData['valence']
                aud_features[index+start][11] = songData['duration_ms']
                aud_features[index+start][12] = songData['speechiness']
                 
            except:
                print("NO METADATA")


    track_data = pd.DataFrame({
    'preview_url': preview_urls_all,
    'id': track_ids_all,
    'popularity': popularity,
    'artists': artists_all,
    'names': names_all,
    'acousticness': aud_features[:,0],
    'danceability': aud_features[:,1],
    'energy': aud_features[:,2],
    'instrumentalness': aud_features[:,3],
    'key': aud_features[:,4],
    'liveness': aud_features[:,5],
    'loudness': aud_features[:,6],
    'mode': aud_features[:,7],
    'tempo': aud_features[:,8],
    'time_signature': aud_features[:,9],
    'valence': aud_features[:,10],
    'duration': aud_features[:,11],
    'speechiness': aud_features[:,12]
    
   
    })
    #q.put(track_data)

    #name = multiprocessing.current_process().name
    print('Exiting')
    return track_data


In [77]:
features_df = get_popularity(296,track_ids=track_ids)

Starting
1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
9001
9101
9201
9301
9401
9501
9601
9701
9801
9901
10001
10101
10201
10301
10401
10501
10601
10701
10801
10901
11001
11101
11201
11301
11401
11501
11601
11701
11801
11901
12001
12101
12201
12301
12401
12501
12601
12701
12801
12901
13001
13101
13201
13301
13401
13501
13601
13701
13801
13901
14001
14101
14201
14301
14401
14501
14601
14701
Exiting


### Examine resulting dataframe

In [78]:
features_df

,acousticness,artists,danceability,duration,energy,id,instrumentalness,key,liveness,loudness,mode,names,popularity,preview_url,speechiness,tempo,time_signature,valence
0,0.987000,Deep Divers,0.239,331843.0,0.01070,1tPGSiwtvmMVAqmniJ7oHp,0.916000,7.0,0.0912,-29.827,0.0,Sea Of Silence,39,https://p.scdn.co/mp3-preview/4b77c40b289080cc...,0.0315,100.031,4.0,0.1380
1,0.201000,Sheryl Crow,0.608,238733.0,0.93800,1tpShM80HMT9vKjRiA3EzW,0.004070,2.0,0.9660,-4.544,1.0,Merry Christmas Baby - Live,12,https://p.scdn.co/mp3-preview/052fb458e56162f4...,0.1980,96.064,4.0,0.7560
2,0.247000,RedOne,0.690,207720.0,0.70200,1TPUbLdzpfgF7wrPEXIAAB,0.000000,11.0,0.2620,-6.103,1.0,Don't You Need Somebody (feat. Enrique Iglesia...,64,https://p.scdn.co/mp3-preview/7e91506026ba3900...,0.0901,92.996,4.0,0.5920
3,0.861000,John Moreland,0.389,254619.0,0.20800,1tQGRq2WOBXjL3JWdWMONg,0.000008,2.0,0.1920,-11.669,1.0,You Don't Care for Me Enough to Cry,40,https://p.scdn.co/mp3-preview/c305d06bef43a04f...,0.0359,90.346,4.0,0.2270
4,0.037200,Placebo,0.387,254760.0,0.89800,1TqmZVIyeD4aN4Wa0wbnfy,0.000040,8.0,0.1100,-6.835,1.0,Pure Morning,54,https://p.scdn.co/mp3-preview/42be861caeeb76ac...,0.0675,81.354,4.0,0.1970
5,0.725000,Good Morning,0.645,240000.0,0.92200,1TquBahz02HCaxj8jy5tTo,0.877000,4.0,0.1840,-5.333,0.0,Cab Deg,44,https://p.scdn.co/mp3-preview/f3464ab234b07ec3...,0.0289,126.310,4.0,0.8310
6,0.062300,Talking Heads,0.646,259600.0,0.88000,1Tr4K5MU5XYE44umXGDndd,0.157000,2.0,0.3550,-7.753,1.0,Once In A Lifetime,37,https://p.scdn.co/mp3-preview/2fb67a5637d04587...,0.0637,116.916,4.0,0.8040
7,0.000073,Oasis,0.316,463120.0,0.87900,1tRlCCMVLSVQrrN9PPgtf0,0.000176,11.0,0.1030,-3.206,0.0,D'You Know What I Mean?,21,https://p.scdn.co/mp3-preview/721b11ba6bd69378...,0.0422,80.036,4.0,0.2350
8,0.000123,Jay Robinson,0.804,358200.0,0.91200,1TScJLsDWzDVBFpCU5vJDo,0.829000,3.0,0.0681,-7.863,0.0,Heiss,20,https://p.scdn.co/mp3-preview/71d988f627707a18...,0.0592,123.993,4.0,0.6000
9,0.872000,Aretha Franklin,0.161,200600.0,0.40900,1TSjS0q24cPW2S4pYj2wEK,0.000679,8.0,0.7050,-9.532,1.0,Nessun Dorma,26,https://p.scdn.co/mp3-preview/d23983c39b4653c2...,0.0335,80.348,4.0,0.1100


### Output resulting data frame to a csv

In [79]:
features_df.to_csv('song_features_laura.csv', encoding='utf-8')

In [46]:
features_df[features_df['duration']>500000]

,acousticness,artists,danceability,duration,energy,id,instrumentalness,key,liveness,loudness,mode,names,popularity,preview_url,tempo,time_signature,valence
34,0.7010,Davidji,0.4800,855387.0,0.3370,1u4yUGqtLb7xFzl38qeW55,0.000015,2.0,0.0859,-15.569,1.0,Beginning Your Journey,16,https://p.scdn.co/mp3-preview/63dc091b107efa7a...,77.033,4.0,0.3840
42,0.9930,Thomas Tallis,0.0751,510333.0,0.0693,1Ud64pWX63TPOgTqoZlkIZ,0.098200,7.0,0.1110,-29.366,1.0,The Lamentations: First Set,26,https://p.scdn.co/mp3-preview/34bd23b19dd8bef5...,78.147,4.0,0.0382
74,0.7010,Davidji,0.4800,855387.0,0.3370,1u4yUGqtLb7xFzl38qeW55,0.000015,2.0,0.0859,-15.569,1.0,Beginning Your Journey,16,https://p.scdn.co/mp3-preview/63dc091b107efa7a...,77.033,4.0,0.3840
82,0.9930,Thomas Tallis,0.0751,510333.0,0.0693,1Ud64pWX63TPOgTqoZlkIZ,0.098200,7.0,0.1110,-29.366,1.0,The Lamentations: First Set,26,https://p.scdn.co/mp3-preview/34bd23b19dd8bef5...,78.147,4.0,0.0382
114,0.7010,Davidji,0.4800,855387.0,0.3370,1u4yUGqtLb7xFzl38qeW55,0.000015,2.0,0.0859,-15.569,1.0,Beginning Your Journey,16,https://p.scdn.co/mp3-preview/63dc091b107efa7a...,77.033,4.0,0.3840
122,0.9930,Thomas Tallis,0.0751,510333.0,0.0693,1Ud64pWX63TPOgTqoZlkIZ,0.098200,7.0,0.1110,-29.366,1.0,The Lamentations: First Set,26,https://p.scdn.co/mp3-preview/34bd23b19dd8bef5...,78.147,4.0,0.0382
143,0.6670,Tim Piggott Smith,0.6260,1048733.0,0.2280,1uKTddJWp8sTcSJ7mT6SjE,0.000000,7.0,0.3680,-23.905,0.0,"Dr Jeckyll & Mr Hyde: Chapter 5, Confrontation",7,https://p.scdn.co/mp3-preview/9ee4ec1d3ae8deb4...,82.618,4.0,0.4570
154,0.7010,Davidji,0.4800,855387.0,0.3370,1u4yUGqtLb7xFzl38qeW55,0.000015,2.0,0.0859,-15.569,1.0,Beginning Your Journey,16,https://p.scdn.co/mp3-preview/63dc091b107efa7a...,77.033,4.0,0.3840
162,0.9930,Thomas Tallis,0.0751,510333.0,0.0693,1Ud64pWX63TPOgTqoZlkIZ,0.098200,7.0,0.1110,-29.366,1.0,The Lamentations: First Set,26,https://p.scdn.co/mp3-preview/34bd23b19dd8bef5...,78.147,4.0,0.0382
183,0.6670,Tim Piggott Smith,0.6260,1048733.0,0.2280,1uKTddJWp8sTcSJ7mT6SjE,0.000000,7.0,0.3680,-23.905,0.0,"Dr Jeckyll & Mr Hyde: Chapter 5, Confrontation",7,https://p.scdn.co/mp3-preview/9ee4ec1d3ae8deb4...,82.618,4.0,0.4570


In [7]:
if __name__ == '__main__':
    q = Queue()
    for offset in [0,500,1000,1500,2000]:
    #for offset in [0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10000,10500,11000,11500,12000,12500,13000,13500,14000,14500]:
        Process(target=get_popularity, args=(offset,10,q, track_ids)).start()
    #p.start()
    #print(q.get())    # prints "[42, None, 'hello']"
    #p.join()

Starting
Starting
Starting
0
0
Starting
Starting
0
0
0


In [ ]:
all_dfs = []
for _ in range(5):
    all_dfs.append(q.get())

In [32]:
playlist_data = all_dfs[0]
for each in all_dfs[1:]:
    playlist_data = playlist_data.append(each,ignore_index=True)
playlist_data.head()

,artists,id,names,popularity,preview_url
0,"[{u'name': u'Deep Divers', u'external_urls': {...",1tPGSiwtvmMVAqmniJ7oHp,Sea Of Silence,39,https://p.scdn.co/mp3-preview/4b77c40b289080cc...
1,"[{u'name': u'Sheryl Crow', u'external_urls': {...",1tpShM80HMT9vKjRiA3EzW,Merry Christmas Baby - Live,12,https://p.scdn.co/mp3-preview/052fb458e56162f4...
2,"[{u'name': u'RedOne', u'external_urls': {u'spo...",1TPUbLdzpfgF7wrPEXIAAB,Don't You Need Somebody (feat. Enrique Iglesia...,64,https://p.scdn.co/mp3-preview/7e91506026ba3900...
3,"[{u'name': u'John Moreland', u'external_urls':...",1tQGRq2WOBXjL3JWdWMONg,You Don't Care for Me Enough to Cry,40,https://p.scdn.co/mp3-preview/c305d06bef43a04f...
4,"[{u'name': u'Placebo', u'external_urls': {u'sp...",1TqmZVIyeD4aN4Wa0wbnfy,Pure Morning,54,https://p.scdn.co/mp3-preview/42be861caeeb76ac...
